# GIS tool for vulnerability assessment of electric infrastructure to extreme weather events
## Maynard J and Mutua B

### Summary/Purpose of Tool:
The following notebook describes the script developed for use as a tool to enable the user to assess the vulnerability of electrical infrastructure to extreme weather events using spatial analysis. Specifically, we seek to enable users to identify generation, transmission, substation and electric vehicle charging assets that may be affected by extreme weather events such as flooding, wildfires, sea-level rise or extreme heat in the State of California. Additionally, we evaluate how accessibility of infrastructure might be impacted during flooding and wildfire events, adapting the methodology used by Jones et al. (2022) to estimate power line restoration access vulnerabilities after a hurricane in Puerto Rico. (https://www.researchgate.net/publication/362089738_Geospatial_Assessment_Methodology_to_Estimate_Power_Line_Restoration_Access_Vulnerabilities_After_a_Hurricane_in_Puerto_Rico)


https://www.energy.gov/sites/prod/files/2019/09/f67/Oak%20Ridge%20National%20Laboratory%20EIS%20Response.pdf

Explore wildfires, extreme heat (max, median & min), sea-level rise, and flooding.
Infrastructure: generation, substations, transmission lines, & EV chargers.
Counties and population groups most affected?

Identify which infrastructure is impacted?
Graph lost capacity for generation plants; derated capacity for transmission lines
Evaluate accessibility after flooding using methodology in Puerto Rico paper 


#### Importing the necessary libraries and setting up relative paths

In [2]:
#Importing necessary libraries
import pandas as pd
#import geopandas as gpd
#import geopandas as gpd
from arcgis.features import FeatureLayer
from arcgis.features import GeoAccessor


#Import the Path function
from pathlib import Path

In [3]:
#Reveal the current working directory
root_folder = Path.cwd().parent
data_folder = root_folder / 'Data'
scratch_folder = root_folder / 'Scratch'
scripts_folder = root_folder / 'Scripts'
print(root_folder)
print(scratch_folder)

v:\Maynard_Mutua
v:\Maynard_Mutua\Scratch


#### Importing FEMA flood plain data: 100 year, 200 year and 500 year

Note: there are several layers available for the 100 year, 200 year and 500 year flood plains. To add an extra data wrangling step, we import all the available layers and dissolve them in one layer for each category below.

##### Importing and combining 100 year flood plain data

In [4]:
#Setting urls for the FEMA feature layers - to change to for loop later

#100 year flood layers
floodrisk_100_year_FEMA = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/5' #FEMA Effective
floodrisk_100_year_Regional = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/7' #Regional/Special Studies
floodrisk_100_year_DWR = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/8' #DWR Awareness
floodrisk_100_year_USACE = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/9' #USACE Comprehensive Study

#200 year flood layer
floodrisk_200_year_USACE = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/11'

#500 year flood layers
floodrisk_500_year_FEMA = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/13' #FEMA Effective
floodrisk_500_year_Regional = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/15' #Regional/Special Studies
floodrisk_500_year_USACE = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/16' #USACE Comprehensive Study




In [4]:
#Read the data in as feature layers
floodrisk_100yearlayer1 = FeatureLayer(floodrisk_100_year_FEMA)
floodrisk_100yearlayer2 = FeatureLayer(floodrisk_100_year_Regional)
floodrisk_100yearlayer3 = FeatureLayer(floodrisk_100_year_DWR)
floodrisk_100yearlayer4 = FeatureLayer(floodrisk_100_year_USACE)

In [5]:
hundredYearFeatureLayers = [floodrisk_100yearlayer1, floodrisk_100yearlayer2, floodrisk_100yearlayer3, floodrisk_100yearlayer4]

for layer in hundredYearFeatureLayers:
    for f in layer.properties.fields:
        print(f['name'])
    print('---------------')

OBJECTID
Shape
DFIRM_ID
VERSION_ID
FLD_AR_ID
STUDY_TYP
FLD_ZONE
ZONE_SUBTY
SFHA_TF
STATIC_BFE
V_DATUM
DEPTH
LEN_UNIT
VELOCITY
VEL_UNIT
AR_REVERT
AR_SUBTRV
BFE_REVERT
DEP_REVERT
DUAL_ZONE
SOURCE_CIT
GFID
Shape_Length
Shape_Area
CO_FIPS
County
---------------
OBJECTID
Shape
FLOODPLAIN
DESCRIPTON
ZONE
AREA
PERIMETER
REGULATION
SourceName
Area_ac
Watercours
DocName
DESCRPTON
CLASS
Entity
Handle
Level
LvlDesc
LyrOn
LvlPlot
Color
Linetype
LyrLnType
Elevation
GGroup
Fill
LineWt
LyrLineWt
LTScale
DocPath
DocType
DocVer
MsLink_DMR
MsCtlg_DMR
Regualtion
Shape_Length
Shape_Area
---------------
OBJECTID
Shape
Source
Folder
Shape_Length
Shape_Area
---------------
OBJECTID_1
Shape
OBJECTID
LAYER
COUNT_
SUM_ACRES
AREA
PERIMETER
ACRES
Shape_Leng
Shape_Length
Shape_Area
---------------


In [6]:
#Import system modules
import arcpy

#Set workspace
arcpy.env.workspace = str(scratch_folder)
arcpy.env.overwriteOutput = True

#Add code to merge or dissolve layers into one
addSourceInfo = 'ADD_SOURCE_INFO'

# Create FieldMappings object to manage merge output fields
fieldMappings = arcpy.FieldMappings()

# Add all fields from both oldStreets and newStreets
fieldMappings.addTable(floodrisk_100_year_FEMA)
fieldMappings.addTable(floodrisk_100_year_Regional)
fieldMappings.addTable(floodrisk_100_year_DWR)
fieldMappings.addTable(floodrisk_100_year_USACE)

# Remove all output fields from the field mappings, except fields 
# 'Shape', 'Shape_Length', & 'Shape_Area'
for field in fieldMappings.fields:
    if field.name not in ['Shape', 'Shape_Length', 'Shape_Area']:
        fieldMappings.removeFieldMap(fieldMappings.findFieldMapIndex(field.name))

# Use Merge tool to move features into single dataset
combined100YearFlood = str(scratch_folder / 'combined100YearFlood') 
arcpy.management.Merge([floodrisk_100_year_FEMA, 
                        floodrisk_100_year_Regional, 
                        floodrisk_100_year_DWR,
                        floodrisk_100_year_USACE],combined100YearFlood, '', addSourceInfo)

# Display any messages, warnings or errors
print(arcpy.GetMessages())

Start Time: Tuesday, December 10, 2024 3:18:19 PM
Succeeded at Tuesday, December 10, 2024 3:22:22 PM (Elapsed Time: 4 minutes 3 seconds)


In [7]:
arcpy.ListFields(floodrisk_100_year_DWR)

[<Field object at 0x2647f8fa990[0x2647f93d5b0]>,
 <Field object at 0x2647f9acd10[0x2647f988af0]>,
 <Field object at 0x2647f9aff90[0x2647f989b70]>,
 <Field object at 0x2647f9afed0[0x2647f989b90]>,
 <Field object at 0x2647f9afc50[0x2647f989bb0]>,
 <Field object at 0x2647f9af5d0[0x2647f989c10]>]

##### Importing and combining 200 year flood plain data

In [8]:
#Repeat for 200 year flood plain data

arcpy.conversion.ExportFeatures(floodrisk_200_year_USACE, str(scratch_folder / 'floodrisk_200_year_USACE'))

<Result 'v:\\Maynard_Mutua\\Scratch\\floodrisk_200_year_USACE.shp'>

##### Importing and combining 500 year flood plain data

In [9]:
#Repeat for 500 year flood plain data

floodrisk_500_year_FEMA = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/13' #FEMA Effective
floodrisk_500_year_Regional = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/15' #Regional/Special Studies
floodrisk_500_year_USACE = 'https://gis.water.ca.gov/arcgis/rest/services/Boundaries/BAM/MapServer/16' #USACE Comprehensive Study

floodrisk_500yearlayer1 = FeatureLayer(floodrisk_500_year_FEMA)
floodrisk_500yearlayer2 = FeatureLayer(floodrisk_500_year_Regional)
floodrisk_500yearlayer3 = FeatureLayer(floodrisk_500_year_USACE)

# Create FieldMappings object to manage merge output fields
fieldMappings = arcpy.FieldMappings()

# Add all fields from both oldStreets and newStreets
fieldMappings.addTable(floodrisk_500_year_FEMA)
fieldMappings.addTable(floodrisk_500_year_Regional)
fieldMappings.addTable(floodrisk_500_year_USACE)

# Remove all output fields from the field mappings, except fields 
# 'Shape', 'Shape_Length', & 'Shape_Area'
for field in fieldMappings.fields:
    if field.name not in ['Shape', 'Shape_Length', 'Shape_Area']:
        fieldMappings.removeFieldMap(fieldMappings.findFieldMapIndex(field.name))

# Use Merge tool to move features into single dataset
combined500YearFlood = str(scratch_folder / 'combined500YearFlood') 
arcpy.management.Merge([floodrisk_500_year_FEMA, 
                        floodrisk_500_year_Regional,
                        floodrisk_500_year_USACE],combined500YearFlood, '', addSourceInfo)

# Display any messages, warnings or errors
print(arcpy.GetMessages())

Start Time: Tuesday, December 10, 2024 3:22:27 PM
Succeeded at Tuesday, December 10, 2024 3:24:27 PM (Elapsed Time: 1 minutes 59 seconds)


##### Convert to spatial dataframes and wrangle data

In [10]:
#Convert layers to spatial dataframes
sdf_floodrisk_100year = GeoAccessor.from_layer(floodrisk_100yearlayer1)
#sdf_floodrisk_200year = GeoAccessor.from_layer(floodrisk_200yearlayer)
#sdf_floodrisk_500year = GeoAccessor.from_layer(floodrisk_500yearlayer)

In [11]:
sdf_floodrisk_100year.columns

Index(['OBJECTID', 'DFIRM_ID', 'VERSION_ID', 'FLD_AR_ID', 'STUDY_TYP',
       'FLD_ZONE', 'ZONE_SUBTY', 'SFHA_TF', 'STATIC_BFE', 'V_DATUM', 'DEPTH',
       'LEN_UNIT', 'VELOCITY', 'VEL_UNIT', 'AR_REVERT', 'AR_SUBTRV',
       'BFE_REVERT', 'DEP_REVERT', 'DUAL_ZONE', 'SOURCE_CIT', 'GFID',
       'Shape_Length', 'Shape_Area', 'CO_FIPS', 'County', 'SHAPE'],
      dtype='object')

In [12]:
sdf_floodrisk_100year.head()

,OBJECTID,DFIRM_ID,VERSION_ID,FLD_AR_ID,STUDY_TYP,FLD_ZONE,ZONE_SUBTY,SFHA_TF,STATIC_BFE,V_DATUM,...,BFE_REVERT,DEP_REVERT,DUAL_ZONE,SOURCE_CIT,GFID,Shape_Length,Shape_Area,CO_FIPS,County,SHAPE
0,1,06087C,1.1.1.0,06087C_6,NP,A,<NA>,T,-9999.0,<NA>,...,-9999.0,-9999.0,<NA>,06087C_STUDY1,5c32f5e6-e407-499e-9aac-21070df175a9,2480.178083,40344.798174,087,Santa Cruz,"{""rings"": [[[-189089.44590000063, -93566.90960..."
1,2,06087C,1.1.1.0,06087C_12,NP,A,<NA>,T,-9999.0,<NA>,...,-9999.0,-9999.0,<NA>,06087C_STUDY1,5c32f5e6-e407-499e-9aac-21070df175a9,724.180157,24847.320406,087,Santa Cruz,"{""rings"": [[[-160979.05110000074, -115433.7480..."
2,3,06087C,1.1.1.0,06087C_14,NP,A,<NA>,T,-9999.0,<NA>,...,-9999.0,-9999.0,<NA>,06087C_STUDY1,5c32f5e6-e407-499e-9aac-21070df175a9,1407.449224,137581.273971,087,Santa Cruz,"{""rings"": [[[-143286.5594999995, -122125.10710..."
3,4,06087C,1.1.1.0,06087C_21,NP,A,<NA>,T,-9999.0,<NA>,...,-9999.0,-9999.0,<NA>,06087C_STUDY1,5c32f5e6-e407-499e-9aac-21070df175a9,15655.534418,902558.705077,087,Santa Cruz,"{""rings"": [[[-182445.5996000003, -96833.979199..."
4,5,06087C,1.1.1.0,06087C_62,NP,A,<NA>,T,-9999.0,<NA>,...,-9999.0,-9999.0,<NA>,06087C_STUDY1,5c32f5e6-e407-499e-9aac-21070df175a9,1057.795507,19269.758301,087,Santa Cruz,"{""rings"": [[[-177217.72959999926, -114214.1187..."


##### Import county and roads shapefiles: could potentially enrich data to apply data engineering principles

In [13]:
#Code to import boundary and roads data
primary_roads_url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Transportation/MapServer/2'
primary_roads_layer = FeatureLayer(primary_roads_url)

secondary_roads_url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Transportation/MapServer/3'
secondary_roads_layer = FeatureLayer(secondary_roads_url)

local_roads_url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Transportation/MapServer/8'
local_roads_layer = FeatureLayer(local_roads_url)

In [14]:
server_url = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/Transportation/MapServer/'
tigerweb_map_service = gis.content.search(server_url)[0]
primary_roads_layer = tigerweb_map_service.layers[2] 
primary_roads_sdf = GeoAccessor.from_layer(primary_roads_layer)


NameError: name 'gis' is not defined

In [15]:
secondary_roads_layer = tigerweb_map_service.layers[3] 
secondary_roads_sdf = GeoAccessor.from_layer(secondary_roads_layer)

NameError: name 'tigerweb_map_service' is not defined

In [ ]:
local_roads_layer = tigerweb_map_service.layers[8] 
local_roads_sdf = GeoAccessor.from_layer(local_roads_layer)

##### Import electrical infrastructure data: transmission lines, substations, power plants and EV charging stations

In [16]:
#Code to import electrical infrastructure
transmission_url = 'https://services3.arcgis.com/bWPjFyq029ChCGur/arcgis/rest/services/Transmission_Line/FeatureServer/2'
transmission_layer = FeatureLayer(transmission_url)
transmission_sdf = GeoAccessor.from_layer(transmission_layer)

In [6]:
powerplants_url = 'https://services3.arcgis.com/bWPjFyq029ChCGur/arcgis/rest/services/Power_Plant/FeatureServer/0'
powerplants_layer = FeatureLayer(powerplants_url)
powerplants_sdf = GeoAccessor.from_layer(powerplants_layer)

In [7]:
#Code to import electrical infrastructure
transmission_url = 'https://services3.arcgis.com/bWPjFyq029ChCGur/arcgis/rest/services/Transmission_Line/FeatureServer/2'
transmission_layer = FeatureLayer(transmission_url)
transmission_sdf = GeoAccessor.from_layer(transmission_layer)

In [8]:
powerplants_url = 'https://services3.arcgis.com/bWPjFyq029ChCGur/arcgis/rest/services/Power_Plant/FeatureServer/0'
powerplants_layer = FeatureLayer(powerplants_url)
powerplants_sdf = GeoAccessor.from_layer(powerplants_layer)

In [9]:
solar_footprints_url = 'https://services3.arcgis.com/bWPjFyq029ChCGur/arcgis/rest/services/Solar_Footprints_V2/FeatureServer/0'
solar_footprints_layer = FeatureLayer(solar_footprints_url)
solar_footprints_sdf = GeoAccessor.from_layer(solar_footprints_layer)

##### Import CalAdapt data by year - Allow user to specify and specific year and import the relevant shapefile in the time series

User inputs:

Flood data - 100 year / 200 year / 500 year
Sea level rise - flood scenario: max/med/min - Brian
Wildfire - get the raster file for a specific year based on user input of scenario and model (decide whether to fix model later) - Justin
Extreme heat - look into how to import and join - Brian


##### Wildfires

In [ ]:
#Code to import wildfire data

##### Sea level rise

In [1]:
#Code to import sea level rise data
import requests 
import json
import re

headers = {'ContentType': 'json'}

api = 'http://api.cal-adapt.org/api'
resource = 'rstores'

search_str = 'CoSMoS'
params = {'name': search_str, 'pagesize': 100}
url = '%s/%s/' % (api, resource)

response = requests.get(url, params, headers=headers)
data = response.json()
results = data['results']
for item in results:
    print(item['url'], item['name'])


# Search for conditions in json
period = '2020-2040'
scenario = 'maximum'

filtered_data = [item for item in results if re.search(r'(?=.*?2020-2040)(?=.*maximum)', item['name'])]
print(filtered_data)

for item in filtered_data:
    print(item['url'], item['name'])



https://api.cal-adapt.org/api/rstores/cosmosflooding_2020-2040_max_mosaic_ci/ CoSMoS flooding mosaic of SLR 25 cm and 100 year storm for maximum flood scenario and 2020-2040 time frame for Channel Islands area
https://api.cal-adapt.org/api/rstores/cosmosflooding_2020-2040_max_mosaic_la/ CoSMoS flooding mosaic of SLR 25 cm and 100 year storm for maximum flood scenario and 2020-2040 time frame for Los Angeles area
https://api.cal-adapt.org/api/rstores/cosmosflooding_2020-2040_max_mosaic_mt/ CoSMoS flooding mosaic of SLR 25 cm and 100 year storm for maximum flood scenario and 2020-2040 time frame for Monterey area
https://api.cal-adapt.org/api/rstores/cosmosflooding_2020-2040_max_mosaic_nc/ CoSMoS flooding mosaic of SLR 25 cm and 100 year storm for maximum flood scenario and 2020-2040 time frame for North Coast area
https://api.cal-adapt.org/api/rstores/cosmosflooding_2020-2040_max_mosaic_sb/ CoSMoS flooding mosaic of SLR 25 cm and 100 year storm for maximum flood scenario and 2020-2040 t

In [7]:
from osgeo import gdal
import matplotlib.pyplot as plt
import numpy as np

rastersToCombine = []

for item in filtered_data:
    dataset = gdal.Open(item['image'], gdal.GA_ReadOnly)
    print(dataset.RasterCount)
    # Save the raster to a file
    filename = item['slug']
    output_file = f'{filename}.tif'  # Defining output filename
    rastersToCombine.append(output_file)
    driver = gdal.GetDriverByName("GTiff")  # Specifying format to create copy
    driver.CreateCopy(output_file, dataset,0,['COMPRESS=DEFLATE'])

    # Close the datasets
    dataset = None

    # Reading the raster band as NumPy array
    #band = np.array(dataset.GetRasterBand(0).ReadAsArray())

    # Ploting the raster using Matplotlib
#    plt.imshow(band)
#    plt.colorbar()
#    plt.show()


1
1
1
1
1
1
1
1


In [2]:
from osgeo import gdal
import matplotlib.pyplot as plt
import numpy as np


for item in filtered_data:
    dataset = gdal.OpenEx(item['image'], gdal.GA_ReadOnly)

    md = dataset.GetMetadata('IMAGE_STRUCTURE')

    # Use dict.get method in case the metadata dict does not have a 'COMPRESSION' key
    compression = md.get('COMPRESSION', None)

    print(md)
    # {'COMPRESSION': 'LZW', 'INTERLEAVE': 'BAND'}
    print(compression)

c:\Users\bmm100\AppData\Local\ESRI\conda\envs\gis\Lib\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


{'COMPRESSION': 'DEFLATE', 'INTERLEAVE': 'BAND', 'LAYOUT': 'COG'}
DEFLATE
{'COMPRESSION': 'DEFLATE', 'INTERLEAVE': 'BAND', 'LAYOUT': 'COG'}
DEFLATE
{'COMPRESSION': 'DEFLATE', 'INTERLEAVE': 'BAND', 'LAYOUT': 'COG'}
DEFLATE
{'COMPRESSION': 'DEFLATE', 'INTERLEAVE': 'BAND', 'LAYOUT': 'COG'}
DEFLATE
{'COMPRESSION': 'DEFLATE', 'INTERLEAVE': 'BAND', 'LAYOUT': 'COG'}
DEFLATE
{'COMPRESSION': 'DEFLATE', 'INTERLEAVE': 'BAND', 'LAYOUT': 'COG'}
DEFLATE
{'COMPRESSION': 'DEFLATE', 'INTERLEAVE': 'BAND', 'LAYOUT': 'COG'}
DEFLATE
{'COMPRESSION': 'DEFLATE', 'INTERLEAVE': 'BAND', 'LAYOUT': 'COG'}
DEFLATE


In [14]:
Path.cwd()

WindowsPath('v:/Maynard_Mutua/Scripts')

In [8]:
rastersToCombine

['cosmosflooding_2020-2040_max_mosaic_ci.tif',
 'cosmosflooding_2020-2040_max_mosaic_la.tif',
 'cosmosflooding_2020-2040_max_mosaic_mt.tif',
 'cosmosflooding_2020-2040_max_mosaic_nc.tif',
 'cosmosflooding_2020-2040_max_mosaic_sb.tif',
 'cosmosflooding_2020-2040_max_mosaic_sd.tif',
 'cosmosflooding_2020-2040_max_mosaic_sf.tif',
 'cosmosflooding_2020-2040_max_mosaic_sl.tif']

In [ ]:
import arcpy

# Code to combine the rasters into one file
arcpy.env.workspace = str(scratch_folder)

combinedRasterFilename = f'{period}+{scenario}.tif'

## Mosaic the TIFF images from the different locations to a new TIFF image
arcpy.MosaicToNewRaster_management(rastersToCombine, str(scratch_folder),\
                                   combinedRasterFilename, '', '', '', '1','LAST','FIRST')

In [13]:
data

{'count': 48,
 'next': None,
 'previous': None,
 'results': [{'id': 330255,
   'tileurl': 'https://api.cal-adapt.org/tiles/cosmosflooding_2020-2040_max_mosaic_ci/{z}/{x}/{y}.png',
   'url': 'https://api.cal-adapt.org/api/rstores/cosmosflooding_2020-2040_max_mosaic_ci/',
   'width': 102774,
   'height': 75011,
   'geom': {'type': 'Polygon',
    'coordinates': [[[-120.421194, 32.743187],
      [-118.228543, 32.783896],
      [-118.247778, 34.136734],
      [-120.474645, 34.093894],
      [-120.421194, 32.743187]]]},
   'event': '2020-01-01',
   'srs': 'PROJCS["NAD83 / UTM zone 11N",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-117],PARAMETER["s

#### Extreme heat (if time allows)

### Find out which assets are affected by extreme weather events

We could find the intersection and create a dataframe with a column for derating and put these in a dashboard showing number of assets affected and severity.

In [10]:
# Finding which electrical assets intersect with flood plains - we could change this into a function to enhance efficiency 
import arcpy

arcpy.env.workspace = str(scratch_folder)    

floodscenario = '100year'
asset = 'powerplants' # change to define asset based on input

if asset == 'powerplants':
    target = powerplants_url
elif asset == 'transmission':
    target = transmission_url
elif asset == 'solarplants':
    target == solar_footprints_url
    
if floodscenario == '100year':
    # Process: Find all power plants that intersect with flood plains based on 100 year scenario
    inFeatures = [combined100YearFlood, target]
    intersectOutput = 'powerPlantsFloodRisk100Year'
    arcpy.analysis.Intersect(inFeatures, intersectOutput, '', '', 'point')
elif floodscenario == '200year':
    # Process: Find all power plants that intersect with flood plains based on 200 year scenario
    inFeatures = [str(scratch_folder / 'floodrisk_200_year_USACE'), target]
    intersectOutput = 'powerPlantsFloodRisk300Year'
    arcpy.analysis.Intersect(inFeatures, intersectOutput, '', '', 'point')
elif floodscenario == '500year':
    # Process: Find all power plants that intersect with flood plains based on 500 year scenario
    inFeatures = [combined500YearFlood, target]
    intersectOutput = 'powerPlantsFloodRisk500Year'
    arcpy.analysis.Intersect(inFeatures, intersectOutput, '', '', 'point')


NameError: name 'combined100YearFlood' is not defined

### Find out which transmission lines and EV chargers might be inaccessible/offline following applicable extreme weather events

We could apply the same approach as the Puerto Rico paper and do a special analysis for EV chargers to show how distance to an working charger might be impacted across the different counties.

### Export shapefiles to be visualized in dashboard